In [ ]:
# author: Sandra Munoz Braceras
# created: 2020-11-24
# last modified: 2020-12-05
# purpose: ML models to classify types of a disease

# Importing packages

In [ ]:
#Import all the packages and functions that will be used
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold,GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import seaborn as sns
import matplotlib.pyplot as plt

# Loading and exploring the data:

In [ ]:
# Open the dataset with the RNA-Seq Data of samples from different types of a disease
df = pd.read_csv("data_for_ML_project.csv",index_col=0)
# Explore the dataset
df.head()

In [ ]:
print('The alignment identified',str(df.shape[1]-1),'genes using the hg38 assembly of the human genome.'
     '\nThere are',df.shape[0],'samples from',str(len(df.types.unique())),'different types of the disease:' )
number_per_type = {}
for i in df.types:
    if i in number_per_type.keys():
        number_per_type[i] += 1
    else:
        number_per_type[i] = 1
for k, v in number_per_type.items():
    print(v,'of',k,', ', end='')

In [ ]:
# From the dataframe, create an array with the unlabelled data (counts of gene reads) 
# and other with the labels (types of disease)
df_data = df.drop(columns=["types"])
np_original_data = df_data.values
df_labels = df["types"]
np_labels = df_labels.values

# Defined functions 
These functions will be used later for the random forest and the svm model to visualize the scores and parameters of the different models creted by GridSearchCV and to visualize the performance of the best model. 

In [ ]:
def selecting_best(clf):
    '''visualize the scores and parameters of the different models creted by GridSearchCV'''
    print('SUMMARY OF RESULTS WITH DIFFERENT PARAMETERS:')
    all_param = clf.cv_results_['params']
    all_means = clf.cv_results_['mean_test_score']
    all_stds = clf.cv_results_['std_test_score']
    for param, mean, std in zip(all_param, all_means, all_stds):
        print(param,'->',str(round(mean,2)),'+/-',str(round(std,2)))
    print('\n the best parameters are:',clf.best_params_,'\n')

In [ ]:
def performance_best(clf, y, X, train_or_test):
    '''visualize the performance of the best model'''
    print('PERFORMANCE OF THE BEST MODEL:')
    y_true, y_pred = y, clf.predict(X)
    print(classification_report(y_true, y_pred))
    # evaluate the performance of the best model
    best_model = clf.best_estimator_
    if train_or_test == 'test':
        print('Confusion matrix for the testing set:')
    elif train_or_test == 'train':
        print('Confusion matrix for the training set:')
    confusion_matrices(y_true,y_pred)

In [ ]:
def confusion_matrices(true_labels, predicted_labels):
    '''display confusion matrices'''
    fig, axes = plt.subplots(1, 2, figsize=(9, 3))
    for i in ['true','pred']:
        if i == 'true':
            j = 0
            title = 'Recall scores in diagonal'
        elif i == 'pred':
            j = 1
            title = 'Precision scores in diagonal' 
        cm = confusion_matrix(true_labels,predicted_labels, normalize=i)
        df_cm = pd.DataFrame(cm,index=['1','2','3','4','5'],columns=['1','2','3','4','5'])
        sns.heatmap(df_cm, annot=True,ax=axes[j], cmap='gist_heat', center=0.5)
        axes[j].set_title(title,fontsize = 11, fontweight = 'bold')
        axes[j].set_ylabel('true types')
        axes[j].set_xlabel('predicted types')
    plt.show()

 # Random Forest Model

In [ ]:
'''Create a Random Forest with the original data (no need for normalization or dim reduction)'''

In [ ]:
# Several parameters for the Random Forest model will be tested 
# by a GridSearch with Cross Validation 
rf_hyperpar = {
    'n_estimators': [65, 95, 125], #between 64 and 128 suggested
    'max_depth': [15, 30],
    'max_features': [2000, 'sqrt'],
    'class_weight':['balanced'],
    'max_samples': [0.5, 0.9],
    'random_state':[0]
    }

# Create training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(np_original_data, np_labels, test_size=0.20, stratify=np_labels, random_state=0)

# Search of the best parameters 
rf_clf = GridSearchCV(RandomForestClassifier(), rf_hyperpar, scoring='f1_weighted', cv = 5, verbose = 3, n_jobs = -1)
rf_clf.fit(X_train, y_train)

selecting_best(rf_clf)

In [ ]:
performance_best(rf_clf,y_test,X_test,'test')

In [ ]:
performance_best(rf_clf,y_train,X_train,'train')

## Identification of the most important genes for the Random Forest Classifier

In [ ]:
# get an idea of the 10 most important genes for the classification
best_model = rf_clf.best_estimator_
important_features = best_model.feature_importances_
indices = np.argsort(important_features)[::-1]
genes= []
importances=[]

for f in range(20):
    genes.append('gene_'+str(indices[f]+1))
    importances.append(important_features[indices[f]])
sns.barplot(y=genes, x=importances, orient='h', palette = 'Blues_r')
sns.despine()
plt.xlabel('Importance')
plt.title('20 most important genes for the model classification',
        fontsize = 12, fontweight = 'bold', y=1.01)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(16, 3))
for i in range(5):
    sns.boxplot(ax=axes[i], x='types', y=genes[i], data=df, order=['type_1','type_2','type_3','type_4','type_5'], palette = 'Spectral')
    axes[i].set_ylabel('Expression levels')
    axes[i].set_xlabel('Disease types')
    axes[i].set_xticks([0,1,2,3,4])
    axes[i].set_xticklabels([1,2,3,4,5])
    axes[i].set_title(genes[i],fontsize = 10, fontweight = 'bold', y=0.99)
    sns.despine()
fig.suptitle('5 most important genes for the model classification',
        fontsize = 12, fontweight = 'bold', y=1.05)
plt.tight_layout()
plt.show()

# Normalizaton of the data for models other than Random Forest

In [ ]:
# Normalize data
scaler = MinMaxScaler()
np_normalized_data = scaler.fit_transform(np_original_data)

# Dimensionality reduction 

In [ ]:
# Dimensionality reduction using PCA to reduce the 59366 features (genes) 
# to a few principal components that describe x% of the variance'''
# I use PCA instead of UMAP because I'll be interested in the interpretation of the results
for i in [0.2,0.5,0.8,0.9,0.95,0.99,0.999]:
    dim_reducer = PCA(n_components=i,svd_solver='full', random_state=0)
    np_reduced_data = dim_reducer.fit_transform(np_normalized_data)
    print(str(np_reduced_data.shape[1]),'principal components are needed to explain', str(i*100),'% of the variance')

# SVM model 

In [ ]:
# Several parameters for the Random Forest model will be tested 
# by a GridSearch with Cross Validation 
for i in [15,138]:
    dim_reducer = PCA(n_components=i,svd_solver='full', random_state=0)
    np_reduced_data = dim_reducer.fit_transform(np_normalized_data)
    print('-------SVM MODEL FOR',i,'DIMENSIONS------')

    svm_hyperpar = {'kernel': ['linear', 'rbf'], 'class_weight' : ['balanced'], 'C' : [0.001,100],'random_state':[0]}

    # Create training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(np_reduced_data, np_labels, test_size=0.20, stratify=np_labels, random_state=0)
    # Search of the best parameters 
    svm_clf = GridSearchCV(SVC(), svm_hyperpar, scoring='f1_weighted', cv = 5, verbose = 1, n_jobs = -1)
    svm_clf.fit(X_train, y_train)

    selecting_best(svm_clf)
    performance_best(svm_clf,y_test,X_test,'test')
    performance_best(svm_clf,y_train,X_train,'train')

# Artificial Neural Network

In [ ]:
from numpy.random import seed
seed(0)

# Transform the labels data to a binary one-hot encoding to be used by the network
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(df_labels)
encoded_labels = np_utils.to_categorical(encoded_labels)

# defining the cross validation method by a variation of KFold that returns stratified folds

for i in [15, 138]:
    dim_reducer = PCA(n_components=i,svd_solver='full', random_state=0)
    np_reduced_data = dim_reducer.fit_transform(np_normalized_data)
    print('-------ANN FOR',i,'DIMENSIONS------')   
    # creating the ANN
    #lists to build the heatmap
    total_labels = []
    total_predictions = []
        
    for j in range(5):
        ann_model = Sequential()
        ann_model.add(Dense(500, input_dim=i, activation='relu'))
        ann_model.add(Dense(200, activation='relu'))
        ann_model.add(Dense(5, activation='softmax'))
        ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
        

        X_train, X_test, y_train, y_test = train_test_split(
            np_reduced_data, encoded_labels, test_size=0.20, stratify=encoded_labels, random_state=0)

        ann_model.fit(X_train, y_train, epochs=10, batch_size=20)
        predictions = ann_model.predict_classes(X_test)
        total_predictions.extend(encoder.inverse_transform(predictions))

        y_test = [np.argmax(y, axis=None, out=None) for y in y_test]
        total_labels.extend(encoder.inverse_transform(y_test))
    
    confusion_matrices(total_labels,total_predictions)